<a href="https://colab.research.google.com/github/hiyuchang/MATH5470-SML/blob/main/QDA_vowel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting, download the data from https://web.stanford.edu/~hastie/ElemStatLearn/ , and put train.data and test.data besides this notebook.

In [9]:
%matplotlib inline
import pandas as pd
train_file = './vowel.train.txt'
train_dat = pd.read_table(train_file,sep=',',index_col=0)
train_dat.columns = ['y', 'x01' , 'x02', 'x03', 'x04', 'x05', 'x06', 'x07', 'x08', 'x09', 'x10']
col_xs = train_dat.columns.values[1:]
train_grp = train_dat.groupby('y')
train_means = train_grp.mean()
train_cov = train_grp.cov()
num_class = train_means.shape[0]

In [10]:
import numpy as np
from scipy.linalg import solve, det
test_file = './vowel.test.txt'
test_dat = pd.read_table(test_file,sep=',',index_col=0)
test_dat.columns = ['y', 'x01' , 'x02', 'x03', 'x04', 'x05', 'x06', 'x07', 'x08', 'x09', 'x10']
num_xs = len(col_xs)
num_test = test_dat.shape[0]
num_class = train_means.shape[0]
loglkhd = np.zeros([num_class, num_test])
for i in range(num_class):
    offsets = pd.DataFrame.transpose(test_dat[col_xs] - train_means.loc[i+1])
    dist = np.einsum('ij,ij->j', offsets, solve(train_cov.loc[i+1], offsets, sym_pos=True))
    loglkhd[i] = -0.5*(np.log(det(train_cov.loc[i+1])) + num_class * np.log(2.0*np.pi) + dist)
pred = np.argmax(loglkhd, axis=0) + 1
test_dat.insert(1, 'pred', pred)

In [11]:
print('The classification success rate is: ')
print((test_dat['y'] == test_dat['pred']).sum() / num_test)
print('Prediction for each test example')
test_dat

The classification success rate is: 
0.47186147186147187
Prediction for each test example


,y,pred,x01,x02,x03,x04,x05,x06,x07,x08,x09,x10
row.names,,,,,,,,,,,,
1,1,1,-1.149,-0.904,-1.988,0.739,-0.060,1.206,0.864,1.196,-0.300,-0.467
2,2,2,-2.613,-0.092,-0.540,0.484,0.389,1.741,0.198,0.257,-0.375,-0.604
3,3,6,-2.505,0.632,-0.593,0.304,0.496,0.824,-0.162,0.181,-0.363,-0.764
4,4,4,-1.768,1.769,-1.142,-0.739,-0.086,0.120,-0.230,0.217,-0.009,-0.279
5,5,7,-2.671,3.155,-0.514,0.133,-0.964,0.234,-0.071,1.192,0.254,-0.471
...,...,...,...,...,...,...,...,...,...,...,...,...
458,7,5,-3.239,3.083,-1.427,-0.202,-0.282,1.421,0.576,0.068,-0.914,0.147
459,8,9,-3.753,3.605,-0.899,-0.747,-0.401,1.765,0.620,0.754,-0.835,-0.301
460,9,9,-3.980,2.459,0.068,0.023,0.237,1.029,-0.189,0.521,-0.773,-0.500
